In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# ## Plotly plotting support
# import plotly.plotly as py

import plotly.offline as py
py.init_notebook_mode(connected=False)

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

# Introduction

In this lecture we examine the process of data cleaning and Exploratory Data Analysis (EDA).  Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA).  Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from the city of Berkeley's public records.  Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data. 

# Getting the Data

To begin this analysis we want to get data about crimes in Berkeley.  Remarkably, the city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining the:

1. [Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)
1. [Stop Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Stop-Data/6e9j-pj9p)

Fortunately, this data is also relatively well document with detailed descriptions of what it contains.  Here are summaries of the fields in the data:

### Calls Data
<img src="calls_desc.png" width=800px />

### Stop Data
<img src="stops_desc.png" width=800px />




## Most data has bad documentation:

Unfortunately, data is seldom well documented and when it is you may not be able to trust the documentation. It is therefore critical that when we download the data we investigate the fields and verify that it reflects the assumptions made in the documentation.



## Reproducible Data Science

In the interest of **reproducible data science** we will download the data programatically.  We have defined some helper functions in the [utils.py](utils.py) file.  I can then reuse these helper functions in many different notebooks.

In [ ]:
from utils import fetch_and_cache

In [ ]:
help(fetch_and_cache)

I can actually look at the source code in the notebook.

In [ ]:
fetch_and_cache??

Occasionally, you will want to modify code that you have imported.  To reimport those modifications you can either use the python importlib library:

```python
from importlib import reload
reload(utils)
```

or use iPython magic which will intelligently import code when files change:

```python
%load_ext autoreload
%autoreload 2
```

## Downloading the Data

Notice that because I record how I got the data in the notebook, others can reproduce this experiment.  However, it is worth noting that **the data can change**.  We will want to pay attention to file timestamps.

In [ ]:
calls_file = fetch_and_cache("https://data.cityofberkeley.info/api/views/k2nh-s5h5/rows.csv?accessType=DOWNLOAD",
                "calls_for_service.csv")

In [ ]:
stops_file = fetch_and_cache("https://data.cityofberkeley.info/resource/4p7k-drdw.json?$limit=100000",
                "stops.json")

To demonstrated live downloading (of something we won't need in lecture) we can download professor Gonzalez's website.  

In [ ]:
gonzalez_file = fetch_and_cache("https://people.eecs.berkeley.edu/~jegonzal/assets/jegonzal.jpg", 
                                "gonzalez.jpg", 
                                force=True)

# Exploring the data

Now that we have obtained the data we want to understand its:

* **Structure** -- the "shape" of a data file
* **Granularity** -- how fine/coarse is each datum
* **Scope** -- how (in)complete is the data
* **Temporality** -- how is the data situated in time
* **Faithfulness** -- how well does the data capture "reality"



## Structure

Before we even begin to load the data it often helps to understand a little about the high-level structure:

1. How much data do I have?
1. How is it formatted?

### How big is the data?

I often like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

In [ ]:
type(stops_file)

In [ ]:
help(stops_file)

In [ ]:
stops_file.stat()

In [ ]:
calls_file.stat().st_size

All the files are relatively small and we could comfortable examine them in a text editors.  (Personally, I like *sublime* or *emacs* but others may have a different *vi*ew.). 

In listing the files I noticed that the names suggest that they are all text file formats:
* **CSV**: Comma separated values is a very standard table format.
* **JSON**: JavaScript Object Notation is a very standard semi-structured file format used to store nested data.

We will dive into the formats in a moment.  However because these are text data I might also want to investigate the number of lines which often correspond to records.

In [ ]:
from utils import line_count
help(line_count)

In [ ]:
print("Lines in the calls file", line_count(calls_file))

In [ ]:
print("Lines in the stops file", line_count(stops_file))

If your iPython session is running on a [POSIX](https://en.wikipedia.org/wiki/POSIX) compliant machine (e.g., mac/linux) you could also run:

```bash
!wc data/*
```

which would produce (lines, words, characters):

```
   16053   85512  976004 data/calls_for_service.csv
     148    1541   54988 data/gonzalez.jpg
   29850  658525 6123225 data/stops.json
   46051  745578 7154217 total
```

In [ ]:
# !wc data/*

If we wanted to learn more about the command we could use the `man` (short for **manual**) command to read the manual page for `wc` (word count).

#### Additional File Meta-Data

Meta-data is data about the data.  Most filesystems will maintain information about when a file was created, read, or modified.  Below we use the Python `time` library to print the time in the local representation.  Note, time is often measured in [Unix Time](https://en.wikipedia.org/wiki/Unix_time) (time in second since January 1st, 1970).


In [ ]:
import time 
print("Time of creation (seconds since Epoch):", 
      stops_file.stat().st_ctime)
print("Local time of creation:", 
      time.ctime(stops_file.stat().st_ctime))


### What is the file format?  (Can we trust extensions?)

We already noticed that the files end in `csv` and `json` which suggests that these are comma separated and javascript object files respectively.  However, we can't always rely on the naming as this is only a convention.  For example, here we picked the name of the file when downloading based on some hints in the URL.



**Often files will have incorrect extensions or no extension at all.**

Let's assume that these are text files (and do not contain binary encoded data) so we can print a "few lines" to get a better understanding of the file.

In [ ]:
from utils import head
help(head)

In [ ]:
head(calls_file)

**Note:** Because the `gonzalez_file` is a JPEG (image) it is not a character file format and looking at the head of the file isn't going to work:

In [ ]:
head(stops_file)

In [ ]:
# head(gonzalez_file)

### What are some observations about `Calls` data?

1. It appears to be in comma separated value (CSV) format.
1. First line contains the column headings.
1. There are lots of **new-line** `\n` characters:
    * at the ends of lines (delimiting records?)
    * *within records* as part of addresses.
1. There are **"quoted"** strings in the `Block_Location` column:
```
"2500 LE CONTE AVE
Berkeley, CA
(37.876965, -122.260544)"
```
these are going to be difficult.  What are the implications on our earlier line count calculations?

In [ ]:
head(calls_file)

### What are some observations about `Stops` data?

This appears to be a fairly standard JSON file.  We notice that the file appears to contain a description of itself in a field called "meta" (which is presumably short for meta-data).  We will come back to this meta data in a moment but first let's quickly discuss the JSON file format.


In [ ]:
head(stops_file, lines=20)


### A quick note on JSON

[JSON (JavaScript Object Notation)](http://www.json.org) is a common format for exchanging complex structured and semi-structured data. 

```javascript
{
    "field1": "value1",
    "field2": ["list", "of", "values"],
    "myfield3": {"is_recursive": true, "a null value": null}
}
```

A few key points:
* JSON is a recursive format in that JSON fields can also contain JSON objects
* JSON closely matches Python Dictionaries:
```python
d = {
    "field1": "value1",
    "field2": ["list", "of", "values"],
    "myfield3": {"is_recursive": True, "a null value": None}
}
print(d['myfield3'])
```
* Very common in web technologies (... JavaScript)
* Many languages have tools for loading and saving JSON objects




## Loading the Data

We will now attempt to load the data into python.  We will be using the Pandas dataframe library for basic tabular data analysis.  Fortunately, the Pandas library has some relatively sophisticated functions for loading data. 


### Loading the Calls Data

Because the file appears to be a relatively well formatted CSV we will attempt to load it directly and allow the Pandas Library to deduce column headers.  (Always check that first row and column look correct after loading.)

In [ ]:
calls = pd.read_csv(calls_file)
calls.head()

How many records did we get?

In [ ]:
calls.size

### Preliminary observations on the data?

1. `EVENTDT` -- Contain the incorrect time stamp
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Appears to be some encoding of the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.


### Checking that the City and State fields are redundant

We notice that there are city and state columns.  Since this is supposed to be data for the city of Berkeley these columns appear to be redundant.  Let's quickly compute the number of occurences of unique values for these two columns.

In [ ]:
calls.groupby(["City", "State"]).count()

### Decoding day of the week

According to the documentation `CVDOW=0` is Sunday, `CVDOW=1` is Monday, ...,  Therefore we can make a series to decode the day of the week for each record and join that series with the calls data.

In [ ]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="DoW")
dow

In [ ]:
calls['Day'] = dow[calls['CVDOW']].reset_index(drop=True)
calls.head()

We can also accomplish the same task using a join.

In [ ]:
df_dow = pd.DataFrame(dow)
pd.merge(calls, df_dow, left_on='CVDOW', right_index=True).head()

### Cleaning Block Location

The block location contains the GPS coordinates and I would like to use these to analyze the location of each request.  Let's try to extract the GPS coordinates using regular expressions (we will cover regular expressions in future lectures):


In [ ]:
calls['Block_Location'][0]

In [ ]:
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t") 
    # Extract Lat and Lon using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*.\d*)\)",expand=True)
)
calls_lat_lon.head(10)

Double check that all of the records have a latitude and longitude:

In [ ]:
calls_lat_lon.isnull().sum()

The following block of code joins the extracted Latitude and Longitude fields with the calls data.  Notice that we actually drop these fields before joining.  This is to enable repeated invocation of this cell even after the join has been completed. 

In [ ]:
# Remove Lat and Lon if they already existed before (reproducible)
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
# Join in the the latitude and longitude data
calls = calls.merge(calls_lat_lon, left_index=True, right_index=True)
# calls[["Lat", "Lon"]] = calls_lat_lon
# calls.join(calls_lat_lon)
calls.head()

We can now look at a few of the records that were missing latitude and longitude entries:

In [ ]:
calls[calls['Lat'].isnull()].head()

---
<br/><br/><br/>


## Loading the `stops.json` Data

Python has relatively good support for JSON data since it closely matches the internal python object model.  In the following cell we import the entire JSON datafile into a python dictionary.

In [ ]:
import json

with open("data/stops.json", "rb") as f:
    stops_json = json.load(f)

The `stops_json` variable is now a dictionary encoding the data in the file:

In [ ]:
type(stops_json)

---
<br/><br/><br/>

### We can now examine what keys are in the top level json object

We can list the keys to determine what data is stored in the object.

In [ ]:
stops_json.keys()

#### Observation

The JSON dictionary contains a `meta` key which likely refers to meta data (data about the data).  Meta data often maintained with the data and can be a good source of additional information.

---
<br/><br/><br/>

## Digging into Meta Data

We can investigate the meta data further by examining the keys associated with the metadata.

In [ ]:
stops_json['meta'].keys()

The `meta` key contains another dictionary called `view`.  This likely refers to meta-data about a particular "view" of some underlying database.  We will learn more about views as we study SQL later in the class.    

In [ ]:
stops_json['meta']['view'].keys()

Notice that this a nested/recursive data structure.  As we dig deeper we reveal more and more keys and the corresponding data:

```
meta
|-> data
    | ... (haven't explored yet)
|-> view
    | -> id
    | -> name
    | -> attribution 
    ...
```

There is a key called description in the view sub dictionary.  This likely contains a description of the data:

In [ ]:
print(stops_json['meta']['view']['description'])

---
<br/><br/><br/>

### Columns Meta data

Another potentially useful key in the meta data dictionary is the `columns`.  This returns a list:

In [ ]:
type(stops_json['meta']['view']['columns'])

We can brows the list using python:

In [ ]:
for c in stops_json['meta']['view']['columns']:
    print(c["name"], "----------->")
    if "description" in c:
        print(c["description"])
    print("======================================\n")



### Observations?

1. The above meta data tells us a lot about the columns in the data including both column names and even descriptions.  This information will be useful in loading and working with the data.
1. JSON makes it easier (than CSV) to create "self-documented data". 
1. Self documenting data can be helpful since it maintains it's own description and these descriptions are more likely to be updated as data changes. 


### Examining the Data Field

We can look at a few entires in the data field


In [ ]:
stops_json['data'][0:2]

## Building a Dataframe from JSON

In the following block of code we:
1. Translate the JSON records into a dataframe
1. Remove columns that have no metadata description.  This would be a bad idea in general but here we remove these columns since the above analysis suggests that they are unlikely to contain useful information.
1. Examine the top of the table

In [ ]:
# Load the data from JSON and assign column titles
stops = pd.DataFrame(
    stops_json['data'],
    columns=[c['name'] for c in stops_json['meta']['view']['columns']])

stops.head()

## Preliminary Observations

What do we observe so far?

We observe:
1. The `Incident Number` appears to have the year encoded in it.  
1. The `Call Date/Time` Field looks to be formatted in YYYY-MM-DDTHH:MM:SS.  I am guessing T means "Time".
1. `Incident Type` has some weird coding.  This is define in the column meta data: 
    *This is the occurred incident type created in the CAD program.  A code signifies a traffic stop (T), suspicious vehicle stop (1196), pedestrian stop (1194) and bicycle stop (1194B).*
    
    
Recall the description:

### Stop Data
<img src="stops_desc.png" width=800px />



--- 

<br/><br/><br/>

# EDA 

Now that we have loaded our various data files.  Let's try to understand a bit more about the data by examining properties of individual fields.

## EDA on the Calls Data

In [ ]:
calls.head()

---
<br/><br/><br/> 

### Checking that City and State are Redundant

We notice that there are city and state columns.  Since this is supposed to be data for the city of Berkeley these columns appear to be redundant.  Let's quickly compute the number of occurences of unique values for these two columns.

In [ ]:
calls.groupby(["City", "State"]).count()

---
<br/><br/><br/>

### Are Case Numbers unique?

Case numbers are probably used internally to track individual cases and my reference other data we don't have access to.  However, it is possible that multiple calls could be associated with the same case.  Let's see if the case numbers are all unique.

In [ ]:
print("There are", len(calls['CASENO'].unique()), "unique case numbers.")
print("There are", len(calls), "calls in the table.")

Are case numbers assigned consecutively.  

In [ ]:
calls['CASENO'].sort_values().reset_index(drop=True).plot()
plt.xlabel("Ordered Location in Data")
plt.ylabel("Case Number")

I like to use interactive plotting tools so I can hover the mouse over the plot and read the values.  The cufflinks library adds plotly support to Pandas.  

In [ ]:
calls['CASENO'].sort_values().reset_index(drop=True).iplot(
    layout = dict(yaxis=dict(title="Case Number", hoverformat="d"), 
                  xaxis=dict(title="Ordered Location in Data")))

Examining the distribution of case numbers shows a similar pattern

In [ ]:
calls['CASENO'].iplot(kind="hist",bins=100)

### What might we be observing?

One possible explanation is that case numbers were assigned consecutively and then sampled uniformly at different rates for two different periods.  We will be able to understand this better by looking at the dates on the cases.

---
<br/><br/><br/>

## Examining the Date

Given the weird behavior with the case numbers let's dig into the date in which events were recorded.  Notice in this data we have several pieces of date/time information (this is not uncommon):
1. **`EVENTDT`**: This contains the date the event took place.  While it has time information the time appears to be `00:00:00`.  
1. **`EVENTTM`**: This contains the time at which the event took place.
1. **`InDbDate`**: This appears to be the date at which the data was entered in the database.  

In [ ]:
calls.head(3)

When Pandas loads more complex fields like dates it will often load them as strings:

In [ ]:
calls["EVENTDT"][0]

We will want to convert these to dates.  Pandas has a fairly sophisticated function `pd.to_datetime` which is capable of guessing reasonable conversions of dates to date objects. 

In [ ]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

We can verify that the translations worked by looking at a few dates:

In [ ]:
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

We can also extract the time field:

In [ ]:
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

To combine the correct date and correct time field we use the built-in python datetime combine function.

In [ ]:
from datetime import datetime
timestamps = pd.to_datetime(pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1))
timestamps.head()

We now updated calls to contain this additional informations:

In [ ]:
calls['timestamp'] = timestamps
calls.head()

### What time range does the data represent

In [ ]:
calls['timestamp'].min()

In [ ]:
calls['timestamp'].max()

---
<br/><br/><br/>

### Back to the Case Numbers

In [ ]:
(
    calls
        .sort_values('timestamp')
        .iplot(
            mode='markers', size=5,
            x='timestamp', y='CASENO',
            layout = dict(yaxis=dict(title="Case Number", hoverformat="d"), 
                  xaxis=dict(title="Date of Call")))
)

#### Explanation?

Perhaps there are multiple different record books with different numbering schemes?  This might be something worth investigating further.

### A Negative Result...

In the above analysis of case numbers all we can really conclude is that we don't understand how this variable was created.  The patterns in the values suggest that there may be some sampling and it is possibly worth contacting the data provider to learn more.

---
<br/><br/><br/>


### Are there any other interesting temporal patterns

Do more calls occur on a particular day of the week?

In [ ]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow].iplot(kind='bar', yTitle="Count")

### How about temporal patterns within a day?

In [ ]:
ts = calls['timestamp']
minute_of_day = ts.dt.hour * 60 + ts.dt.minute
hour_of_day = minute_of_day / 60.

calls['minute_of_day'] = minute_of_day
calls['hour_of_day'] = hour_of_day


In [ ]:
py.iplot(ff.create_distplot([hour_of_day],group_labels=["Hour"],bin_size=1))

### Observations?

In the above plot we see the standard pattern of limited activity early in the morning around here 6:00AM.

---
<br/><br/><br/>

### Smoothing Parameters

In the above plot we see a smoothed curve approximating the histogram.  This is an example of a kernel density estimator (KDE).  The KDE, like the histogram, has a parameter that determines it's smoothness.  Many packages (Plotly and Seaborn) use a boostrap like procedure to choose the best value.  

To understand how this parameter works we will use seaborn which let's you also set the parameter manually.  

In [ ]:
sns.distplot(hour_of_day, rug=False)

Notice in the above plot that the interpolation tries to follow the histogram to closely. 

### Stratified Analysis

To better understand the time of day a report occurs we could stratify the analysis by the day of the week.  To do this we will use box plots.  


In [ ]:
## Using Pandas built in box plot
# calls['hour_of_day'] = minute_in_day
# calls.boxplot('hour_of_day', by='Day')

In [ ]:
groups = calls.groupby("Day")
boxes = {i: go.Box(y=df["hour_of_day"], name=i) for (i, df) in groups}
py.iplot([boxes[d] for d in dow])

In [ ]:
py.iplot(ff.create_violin(calls, data_header='hour_of_day', group_header="Day"))

#### Observations?

There are no very clear patterns here.  It is possible Fridays might have crimes shifted slightly more to the evening.  In general the time a crime is reported seems fairly consistent throughout the week.

---
<br/><br/><br/>

### Examining the Event

We also have data about the different kinds of crimes being reported

In [ ]:
calls.head()

### The Offense Field

The Offense field appears to contain the specific crime being reported.  As nominal data we might want to see a summary constructed by computing counts of each offense type:

In [ ]:
calls['OFFENSE'].value_counts(dropna=False).iplot(kind="bar")

#### Observations?

Car burglary and misdemeanor theft seem to be the most common crimes with many other types of crimes occurring rarely.

---
<br/><br/><br/>

### CVLEGEND

The CVLEGEND filed provides the broad category of crime and is a good mechanism to group potentially similar crimes. 

In [ ]:
calls['CVLEGEND'].value_counts(dropna=False).iplot(kind="bar")

Notice that when we group by the crime time we see that **larceny** becomes the dominant category.  Larceny is essentially stealing -- taking someone else stuff without force.

### Stratified Analysis of Time of Day by CVLEGEND

View the crime time periods broken down by crime type:

In [ ]:
boxes = [(len(df), go.Box(y=df["hour_of_day"], name=i)) for (i, df) in calls.groupby("CVLEGEND")]
py.iplot([r[1] for r in sorted(boxes, key=lambda x:x[0], reverse=True)])

In [ ]:
py.iplot(ff.create_distplot([
    calls[calls['CVLEGEND'] == "NOISE VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "DRUG VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "LIQUOR LAW VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "FRAUD"]['hour_of_day']
],
    group_labels=["Noise Violation", "Drug Violation", "Liquor Violation", "Fraud"], 
    ))


## Examining Location information

Let's examine the geographic data (latitude and longitude).  Recall that we had some missing values.  Let's look at the behavior of these missing values according to crime type.

In [ ]:
missing_lat_lon = calls[calls[['Lat', 'Lon']].isnull().any(axis=1)]
missing_lat_lon['CVLEGEND'].value_counts().iplot(kind="bar")

### Observations?

There is a clear bias towards drug violations that is not present in the original data.  Therefore we should be careful when dropping missing values!

We might further normalize the analysis by the frequency to find which type of crime has the highest proportion of missing values.

In [ ]:
(
    missing_lat_lon['CVLEGEND'].value_counts() / calls['CVLEGEND'].value_counts()
).sort_values(ascending=False).iplot(kind="bar")

---
<br/><br/><br/> 

### Day of Year and Missing Values:

We might also want to examine the time of day in which latitude and longitude are missing:

In [ ]:
ts = calls['timestamp']

py.iplot(ff.create_distplot([calls['timestamp'].dt.dayofyear, 
                             missing_lat_lon['timestamp'].dt.dayofyear],
                            group_labels=["all data", "missing lat-lon"],bin_size=2))

### Observations?

Pretty similar 

In [ ]:
!pip install --upgrade folium
import folium
print(folium.__version__, "should be at least 0.3.0")

In [ ]:
import folium
import folium.plugins # The Folium Javascript Map Library

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().as_matrix()
heatmap = folium.plugins.HeatMap(locs.tolist(), radius = 10)
sf_map.add_child(heatmap)

### Questions

1. Is campus really the safest place to be?
1. Why are all the calls located on the street and at often at intersections?


In [ ]:
cluster = folium.plugins.MarkerCluster()
for _, r in calls[['Lat', 'Lon', 'CVLEGEND']].tail(1000).dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])], popup=r['CVLEGEND']))
    
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map